# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [2]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions).

In [3]:
# Your code

#response = requests.get('https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale')
#results = response.json()
#results
import urllib.request
uf = urllib.request.urlopen('https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale')
response = uf.read().decode('utf-8')
results = json.loads(response)


flattened_data = json_normalize(results)

flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1

,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,url_key
0,Puma,"[{'sku': 'PU113D04G-C11', 'url_key': 'puma-pla...","[{'key': 'discountRate', 'value': '-50%', 'tra...",False,[{'path': 'PU/11/3D/04/GC/11/PU113D04G-C11@10....,PLATFORM RADICALS - Baskets basses - gray viol...,False,False,False,False,"74,95 €","37,48 €",shoe,"[36, 37, 38, 39]",PU113D04G-C11,puma-platform-radicals-baskets-basses-pu113d04...
1,Timberland,"[{'sku': 'TI116I010-B11', 'url_key': 'timberla...","[{'key': 'discountRate', 'value': 'Jusqu’à -40...",False,[{'path': 'TI/11/6I/01/0B/11/TI116I010-B11@10....,PREMIUM BOOT - Bottines à lacets - wheat,False,True,True,False,"149,95 €","90,00 €",shoe,"[36, 37, 38, 39, 40]",TI116I010-B11,timberland-premium-boot-bottines-a-lacets-whea...
2,Geox,"[{'sku': 'GE114G01K-K11', 'url_key': 'geox-kay...","[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'GE/11/4G/01/KK/11/GE114G01K-K11@3.j...,KAYTAN BOY - Chaussures premiers pas - navy/avio,False,False,False,False,"48,95 €","34,27 €",shoe,"[20, 21, 22, 23, 24, 25]",GE114G01K-K11,geox-kaytan-boy-sandales-navyavio-ge114g01k-k11
3,Jordan,"[{'sku': 'JOC16D005-Q11', 'url_key': 'jordan-j...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'JO/C1/6D/00/5Q/11/JOC16D005-Q11@6.j...,BIG FUND - Baskets montantes - black/metallic ...,False,False,False,False,"94,95 €","76,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",JOC16D005-Q11,jordan-jordan-big-fund-baskets-montantes-joc16...
4,Jordan,"[{'sku': 'JOC16D00B-Q11', 'url_key': 'jordan-l...","[{'key': 'discountRate', 'value': 'Jusqu’à -40...",False,[{'path': 'JO/C1/6D/00/BQ/11/JOC16D00B-Q11@9.j...,LEGACY 312 - Baskets montantes - black/varsit...,False,True,True,False,"74,95 €","45,00 €",shoe,"[27.5, 28, 28.5, 29.5, 30, 31, 31.5, 32, 33, 3...",JOC16D00B-Q11,jordan-legacy-312-baskets-montantes-blackvarsi...
5,Nike Sportswear,"[{'sku': 'NI114D07D-A12', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NI/11/4D/07/DA/12/NI114D07D-A12@11....,CORTEZ BASIC - Baskets basses - white / varsi...,False,False,False,False,"64,95 €","52,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",NI114D07D-A12,nike-sportswear-cortez-basic-baskets-basses-wh...
6,Polo Ralph Lauren,"[{'sku': 'PO223J01P-K11', 'url_key': 'polo-ral...","[{'key': 'discountRate', 'value': '-20%', 'tra...",True,[{'path': 'PO/22/3J/01/PK/11/PO223J01P-K11@7.j...,T-shirt à manches longues - carson blue,False,True,True,False,"32,95 €","26,36 €",clothing,"[2a, 3a, 4a, 5a, S, M, L, XL, 1.5a]",PO223J01P-K11,polo-ralph-lauren-t-shirt-a-manches-longues-ca...
7,Nike Sportswear,"[{'sku': 'NI114D06L-Q11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'NI/11/4D/06/LQ/11/NI114D06L-Q11@12....,CORTEZ BASIC SL (TDV) - Baskets basses - white...,False,True,True,False,"44,95 €","36,00 €",shoe,"[17, 18.5, 19.5, 21, 22, 23.5, 25, 26, 27]",NI114D06L-Q11,nike-sportswear-baskets-basses-whiteprism-pink...
8,adidas Performance,"[{'sku': 'AD543E0JL-Q11', 'url_key': 'adidas-p...","[{'key': 'discountRate', 'value': 'Jusqu’à -25...",False,[{'path': 'AD/54/3E/0J/LQ/11/AD543E0JL-Q11@6.j...,CORE - Pantalon de survêtement - black/white,False,True,True,False,"24,95 €","18,71 €",clothing,"[5-6a, 7-8a, 9-10a, 11-12a, 13-14a, 15-16a]",AD543E0JL-Q11,adidas-performance-core-pantalon-de-survetemen...
9,Havaianas,"[{'sku': 'HA113G008-J11', 'url_key': 'havaiana...","[{'key': 'discountRate', 'value': '-5%', 'trac...",False,[{'path': 'HA/11/3G/00/8J/11/HA113G008-J11@5.j...,TOP MIX - Tongs - rose quartz/ice blue,False,False,False,False,"15,95 €","15,15 €",shoe,"[25/26, 27/28, 29/30, 31/32]",HA113G008-J11,havaianas-top-mix-tongs-rose-quartzice-blue-ha...


#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [4]:
# Get the total number of pages
total_pages=results['pagination']['page_count']

# Your code
df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    flattened_data1 = json_normalize(flattened_data.articles[0])
    flattened_data1=flattened_data1.set_index('sku')
    df = df.append(flattened_data1)

df

,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,url_key
sku,,,,,,,,,,,,,,,
PU113D04G-C11,Puma,"[{'sku': 'PU113D04G-C11', 'url_key': 'puma-pla...","[{'key': 'discountRate', 'value': '-50%', 'tra...",False,[{'path': 'PU/11/3D/04/GC/11/PU113D04G-C11@10....,PLATFORM RADICALS - Baskets basses - gray viol...,False,False,False,False,"74,95 €","37,48 €",shoe,"[36, 37, 38, 39]",puma-platform-radicals-baskets-basses-pu113d04...
TI116I010-B11,Timberland,"[{'sku': 'TI116I010-B11', 'url_key': 'timberla...","[{'key': 'discountRate', 'value': 'Jusqu’à -40...",False,[{'path': 'TI/11/6I/01/0B/11/TI116I010-B11@10....,PREMIUM BOOT - Bottines à lacets - wheat,False,True,True,False,"149,95 €","90,00 €",shoe,"[36, 37, 38, 39, 40]",timberland-premium-boot-bottines-a-lacets-whea...
GE114G01K-K11,Geox,"[{'sku': 'GE114G01K-K11', 'url_key': 'geox-kay...","[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'GE/11/4G/01/KK/11/GE114G01K-K11@3.j...,KAYTAN BOY - Chaussures premiers pas - navy/avio,False,False,False,False,"48,95 €","34,27 €",shoe,"[20, 21, 22, 23, 24, 25]",geox-kaytan-boy-sandales-navyavio-ge114g01k-k11
JOC16D005-Q11,Jordan,"[{'sku': 'JOC16D005-Q11', 'url_key': 'jordan-j...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'JO/C1/6D/00/5Q/11/JOC16D005-Q11@6.j...,BIG FUND - Baskets montantes - black/metallic ...,False,False,False,False,"94,95 €","76,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",jordan-jordan-big-fund-baskets-montantes-joc16...
JOC16D00B-Q11,Jordan,"[{'sku': 'JOC16D00B-Q11', 'url_key': 'jordan-l...","[{'key': 'discountRate', 'value': 'Jusqu’à -40...",False,[{'path': 'JO/C1/6D/00/BQ/11/JOC16D00B-Q11@9.j...,LEGACY 312 - Baskets montantes - black/varsit...,False,True,True,False,"74,95 €","45,00 €",shoe,"[27.5, 28, 28.5, 29.5, 30, 31, 31.5, 32, 33, 3...",jordan-legacy-312-baskets-montantes-blackvarsi...
NI114D07D-A12,Nike Sportswear,"[{'sku': 'NI114D07D-A12', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NI/11/4D/07/DA/12/NI114D07D-A12@11....,CORTEZ BASIC - Baskets basses - white / varsi...,False,False,False,False,"64,95 €","52,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",nike-sportswear-cortez-basic-baskets-basses-wh...
PO223J01P-K11,Polo Ralph Lauren,"[{'sku': 'PO223J01P-K11', 'url_key': 'polo-ral...","[{'key': 'discountRate', 'value': '-20%', 'tra...",True,[{'path': 'PO/22/3J/01/PK/11/PO223J01P-K11@7.j...,T-shirt à manches longues - carson blue,False,True,True,False,"32,95 €","26,36 €",clothing,"[2a, 3a, 4a, 5a, S, M, L, XL, 1.5a]",polo-ralph-lauren-t-shirt-a-manches-longues-ca...
NI114D06L-Q11,Nike Sportswear,"[{'sku': 'NI114D06L-Q11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'NI/11/4D/06/LQ/11/NI114D06L-Q11@12....,CORTEZ BASIC SL (TDV) - Baskets basses - white...,False,True,True,False,"44,95 €","36,00 €",shoe,"[17, 18.5, 19.5, 21, 22, 23.5, 25, 26, 27]",nike-sportswear-baskets-basses-whiteprism-pink...
AD543E0JL-Q11,adidas Performance,"[{'sku': 'AD543E0JL-Q11', 'url_key': 'adidas-p...","[{'key': 'discountRate', 'value': 'Jusqu’à -25...",False,[{'path': 'AD/54/3E/0J/LQ/11/AD543E0JL-Q11@6.j...,CORE - Pantalon de survêtement - black/white,False,True,True,False,"24,95 €","18,71 €",clothing,"[5-6a, 7-8a, 9-10a, 11-12a, 13-14a, 15-16a]",adidas-performance-core-pantalon-de-survetemen...


In [5]:
# Get the total number of pages
total_pages=results['pagination']['page_count']
print(total_pages)

90


#### Display the trending brand in DataFrame

In [6]:
df.brand_name.value_counts().index[0]


'Nike Sportswear'

#### Display the brand with maximal total discount (sum of discounts on all goods)

In [7]:
#Our data is still text. Convert prices into numbers:
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]



In [9]:
df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df['discount_amount']

sku
PU113D04G-C11    37.47
TI116I010-B11    59.95
GE114G01K-K11    14.68
JOC16D005-Q11    18.95
JOC16D00B-Q11    29.95
NI114D07D-A12    12.95
PO223J01P-K11     6.59
NI114D06L-Q11     8.95
AD543E0JL-Q11     6.24
HA113G008-J11     0.80
AD116D009-Q12    20.95
F5713I03E-K11     9.00
NI114D057-Q11     4.49
LE224K01L-Q11     9.95
NI114D0AC-A11    17.95
AD116D07G-A11     9.95
N1243G022-Q11     8.74
AD116D0IE-C11     7.95
KI114G00R-K11     5.95
PU114D03S-K12    19.23
CO414D074-K11     8.95
TO124K042-C11    27.95
PU113D03X-J11    17.48
KI114G00O-K11    16.95
PU143K024-J11    20.00
PU113D052-A11    23.00
NI113D07O-A11    13.95
KI113F01Q-J11    20.95
LA214D03C-A11     5.95
CO413A01B-A11     5.95
                 ...  
GE113D04O-K11    26.95
NI114D03M-Q11     8.95
N1243A0PL-Q11    13.49
NI114D09E-J11    10.95
PU143G04N-G11    16.20
LE224C01E-K11     2.50
RI923B01L-H11     2.70
F5713I03A-J11    12.00
SC624C012-Q11    31.48
LE223L00O-K11     5.95
PO223J01P-I11    13.20
CO416D04O-Q11    11.95
C2723F0

In [8]:
df['price.original'] = pd.to_numeric(df['price.original'], errors='coerce')
df['price.promotional'] = pd.to_numeric(df['price.promotional'], errors='coerce')

In [11]:
total_disc=df.groupby(['brand_name']).sum().discount_amount

In [12]:
total_disc.sort_values(ascending=False).index[0]
print(total_disc)

brand_name
Catimini                    1101.6
Champion                     242.1
Converse                    5687.1
Crocs                        864.0
Easy Peasy                  1795.5
Friboo                      2070.0
Geox                        3746.7
Havaianas                     72.0
Jordan                      4401.0
Kickers                     3946.5
Lacoste                     1435.5
Levi's®                     2731.5
New Look 915 Generation      377.1
Nike Performance            5231.7
Nike SB                     1615.5
Nike Sportswear            22875.3
Pepe Jeans                  1820.7
Polo Ralph Lauren           1781.1
Puma                       13759.2
Reebok Classic              1484.1
River Island                 243.0
Scotch Shrunk               2833.2
Timberland                  8635.5
Tommy Hilfiger              3501.0
adidas Originals           14260.5
adidas Performance          2357.1
Name: discount_amount, dtype: float64


#### Display the brands without discount at all

In [13]:
nodisc = total_disc[total_disc==0]
print(nodisc)

Series([], Name: discount_amount, dtype: float64)


In [14]:
nodisc

Series([], Name: discount_amount, dtype: float64)